In [11]:
import sys
# sys.path.append('/home/abigaile-woodbury/Research/cp_reach')  # UPDATE path as needed
sys.path.append('/home/micah/Research/development/cp_reach')
import json
import numpy as np
import matplotlib.pyplot as plt
import cp_reach as cp

import cp_reach.satellite.invariant_set3 as invariant_set3
import cp_reach.satellite.plotting as plotting
import cp_reach.satellite.HCW as HCW
import cp_reach.satellite.TH_LTV as TH_LTV
import cp_reach.physics.rigid_body as rigid_body
import sympy as sp

import cp_reach.satellite.mission2 as sat_sim
import cyecca.lie as lie
import casadi as ca

In [12]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%autoreload 2

# Define Mission Parameters

In [14]:
thrust = 0.5
ang_vel_dist_deg = 10 # in degrees
ang_vel_dist_rad = np.deg2rad(ang_vel_dist_deg)
gravity_err = 0.00 # m/s^2
accel_dist = 0 #m/s^2
Kp = 1
Kd = 1
Kpq = 1
Kdq = 1
control_gains = (Kp, Kd, Kpq, Kdq)
acc = [thrust, 0, 0]

In [15]:
def do_burn_reference(t_vect):
    sat_burn = sat_sim.SatSimBurn()
    data = []
    p_index = sat_burn.model['p_index']
    sat_burn.model['p0_dict']['w_d_phase'] = 0
    sat_burn.model['p0_dict']['w_d_freq'] = 0
    
    sat_burn.model['p0_dict']['w_d_amp'] = 0
    sat_burn.model['p0_dict']['a_d_amp'] = 0
    sat_burn.model['p0_dict']['thrust'] = thrust
    sat_burn.model['p0_dict']['Kp'] = Kp
    sat_burn.model['p0_dict']['Kd'] = Kd
    sat_burn.model['p0_dict']['Kpq'] = Kpq
    sat_burn.model['p0_dict']['Kdq'] = Kdq
    
    res = sat_burn.simulate(t_vect, integrator='rk')
    return res

def do_burn_monte_carlo(t_vect, n_runs=10):
    sat_burn = sat_sim.SatSimBurn()
    data = []
    for i in range(n_runs):
        p_index = sat_burn.model['p_index']
        sat_burn.model['p0_dict']['w_d_x_phase'] = np.random.uniform(0, 2*np.pi)
        sat_burn.model['p0_dict']['w_d_y_phase'] = np.random.uniform(0, 2*np.pi)
        sat_burn.model['p0_dict']['w_d_z_phase'] = np.random.uniform(0, 2*np.pi)
        sat_burn.model['p0_dict']['w_d_freq'] = 10**np.random.uniform(-2, 1)

        
        sat_burn.model['p0_dict']['w_d_amp'] = ang_vel_dist_rad
        sat_burn.model['p0_dict']['a_d_amp'] = accel_dist
        sat_burn.model['p0_dict']['thrust'] = thrust
        sat_burn.model['p0_dict']['Kp'] = Kp
        sat_burn.model['p0_dict']['Kd'] = Kd
        sat_burn.model['p0_dict']['Kpq'] = Kpq
        sat_burn.model['p0_dict']['Kdq'] = Kdq

        res = sat_burn.simulate(t_vect, integrator='rk')
        data.append(res)
    return data

# 10 Second Burn - Monte Carlo and Reference

In [16]:
# Perform Monte Carlo Simulation for Burn Phase
t_burn_vect = np.arange(0, 3*60*60, 0.01)
data_burn_ref = do_burn_reference(t_burn_vect)
# data_burn = do_burn_monte_carlo(t_vect=t_burn_vect, n_runs=100)

## LMI for log-linearized system

In [17]:
%time
xi_points, eta_points, bounds_log, sol = invariant_set3.solve(ang_vel_dist_rad, accel_dist, acc, control_gains, num_points=1000)

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.53 µs


In [18]:
bounds_log

array([[-0.00869744,  0.00291728],
       [-0.11986819,  0.11986819],
       [-0.11986818,  0.11986818],
       [-0.0064176 ,  0.00352511],
       [-0.09881523,  0.09881523],
       [-0.09881522,  0.09881522],
       [-0.20153381,  0.20153381],
       [-0.20153381,  0.20153381],
       [-0.20736972,  0.20736972]])

In [19]:
# i = 4
# j = 5
# plt.scatter(eta_points[i,:],eta_points[j,:])
# plt.scatter(xi_points[i,:],xi_points[j,:])
# plt.xlabel('deltav y')
# plt.ylabel('deltav z')

## LMI for HCW system

In [20]:
mu = 3.986e14      # [m^3/s^2]
r0 = 42164e3          # [m]
ref_n = np.sqrt(mu / r0**3)

hcw_sol = HCW.solve_hcw_invariant_set(ref_n, accel_dist, Kp=Kp, Kd=Kd)
hcw_sol
mu1 = hcw_sol['mu']  # acceleration disturbance multiplier
P_hcw = hcw_sol['P']
val_hcw = mu1 * accel_dist**2 
P_hcw_scaled = P_hcw / val_hcw
hcw_points = HCW.sample_ellipsoid_boundary_6(P_hcw_scaled, n=600)

hcw_min = np.min(hcw_points, axis=1)
hcw_max = np.max(hcw_points, axis=1)
bounds_hcw = np.column_stack([hcw_min, hcw_max])

/tmp/ipykernel_127424/1265580442.py:10: RuntimeWarning: divide by zero encountered in divide
  P_hcw_scaled = P_hcw / val_hcw


In [21]:
bounds_hcw

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [-0., -0.]])

## LMI for TH system

In [22]:
mu = 3.986e14      # [m^3/s^2]
ref_orbit = data_burn_ref['xf'][1:7,:].T
bounds= TH_LTV.th_param_bounds_from_traj(ref_orbit, mu)
bounds

[7.28109288e-05 7.28110473e-05 7.28111659e-05 ... 7.57371329e-05
 7.57370319e-05 7.57369309e-05] [-0.00000000e+00  2.34587783e-17  4.68799127e-17 ... -1.36226679e-08
 -1.36226541e-08 -1.36226404e-08] 0.003438144185977713


{'a_min': 5.301431346076512e-09,
 'a_max': 1.3517766433699728e-08,
 'b_min': 7.281092875438764e-05,
 'b_max': 0.00011626592980619786,
 'c_min': -1.5297408407270463e-08,
 'c_max': 7.311354740853109e-15,
 'd_min': 2.13608702496548e-09,
 'd_max': 4.310253264768419e-09,
 'h_mean': 237761832341.42194,
 'k': 0.003438144185977713}

In [23]:
%timeit
th_sol = TH_LTV.solve_YA_TH_LTV_invariant_set_from_bounds(bounds, accel_dist,Kp=Kp, Kd=Kd)
print(th_sol)
mu1 = th_sol['mu']  # acceleration disturbance multiplier
P_th = th_sol['P']
val_th = mu1 * accel_dist**2
print(mu1)
P_th_scaled = P_th / val_th
th_points = TH_LTV.sample_ellipsoid_boundary_6(P_th_scaled, n=600)

th_min = np.min(th_points, axis=1)
th_max = np.max(th_points, axis=1)
bounds_th = np.column_stack([th_min, th_max])

{'P': array([[ 4.16777735e+01,  3.85910085e-15,  6.86858058e-11,
         1.36914588e+01, -4.45747128e-15, -8.64846836e-03],
       [ 3.85910085e-15,  4.16777749e+01, -5.36332383e-15,
         4.76348319e-15,  1.36914590e+01, -1.63090854e-14],
       [ 6.86858058e-11, -5.36332383e-15,  4.16777731e+01,
         8.64846833e-03,  2.49987458e-14,  1.36914589e+01],
       [ 1.36914588e+01,  4.76348319e-15,  8.64846833e-03,
         4.87401400e+01, -6.10345168e-16,  1.27467517e-10],
       [-4.45747128e-15,  1.36914590e+01,  2.49987458e-14,
        -6.10345168e-16,  4.87401414e+01, -9.63057798e-15],
       [-8.64846836e-03, -1.63090854e-14,  1.36914589e+01,
         1.27467517e-10, -9.63057798e-15,  4.87401401e+01]]), 'mu': 109.2904512139394, 'cost': 0.0, 'alpha': np.float64(0.49994095254421517), 'prob': Problem(Minimize(Expression(AFFINE, ZERO, ())), [PSD(Expression(AFFINE, UNKNOWN, (6, 6))), PSD(Expression(AFFINE, UNKNOWN, (9, 9))), PSD(Expression(AFFINE, UNKNOWN, (9, 9))), PSD(Expression(

/tmp/ipykernel_127424/3093238923.py:8: RuntimeWarning: divide by zero encountered in divide
  P_th_scaled = P_th / val_th


In [24]:
bounds_th

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [-0., -0.]])

## Plot

In [25]:
# def burn_plotting(data_burn, bounds):
#     fig, axes = plt.subplots(8, 1, figsize=(10, 20))
#     axes[0].set_title('Burn Phase')
#     colors = ['r','g','b']

#     # Attitude
#     plotting.plot_burn_attitude(axes[1], data_burn)
#     for i in range(3):
#         axes[1].axhline(y=np.rad2deg(bounds_log[6+i, 0]), linestyle='--', color=colors[i])
#         axes[1].axhline(y=np.rad2deg(bounds_log[6+i, 1]), linestyle='--', color=colors[i])
#         axes[1].axhline(y=np.rad2deg(bounds_log[6+i, 1]), linestyle='--', color=colors[i])

#     # Vx
#     plotting.plot_burn_delta_vx_error(axes[2], data_burn)
#     axes[2].axhline(y=bounds_log[3, 0], linestyle='--')
#     axes[2].axhline(y=bounds_log[3, 1], linestyle='--')

#     # axes[2].axhline(y=bounds_hcw[3, 0], linestyle='--')
#     # axes[2].axhline(y=bounds_hcw[3, 1], linestyle='--')

#     axes[2].axhline(y=bounds_th[3, 0], linestyle='--')
#     axes[2].axhline(y=bounds_th[3, 1], linestyle='--')
    

#     #V y
#     plotting.plot_burn_delta_vy_error(axes[3], data_burn)
#     axes[3].axhline(y=bounds_log[4, 0], linestyle='--')
#     axes[3].axhline(y=bounds_log[4, 1], linestyle='--')

#     # axes[3].axhline(y=bounds_hcw[4, 0], linestyle='--')
#     # axes[3].axhline(y=bounds_hcw[4, 1], linestyle='--')

#     axes[3].axhline(y=bounds_th[4, 0], linestyle='--')
#     axes[3].axhline(y=bounds_th[4, 1], linestyle='--')

#     # Vz
#     plotting.plot_burn_delta_vz_error(axes[4], data_burn)
#     axes[4].axhline(y=bounds_log[5, 0], linestyle='--')
#     axes[4].axhline(y=bounds_log[5, 1], linestyle='--')

#     # axes[4].axhline(y=bounds_hcw[5, 0], linestyle='--')
#     # axes[4].axhline(y=bounds_hcw[5, 1], linestyle='--')

#     axes[4].axhline(y=bounds_th[5, 0], linestyle='--')
#     axes[4].axhline(y=bounds_th[5, 1], linestyle='--')

#     # Px
#     plotting.plot_burn_x_error(axes[5], data_burn)
#     axes[5].axhline(y=bounds_log[0, 0], linestyle='--')
#     axes[5].axhline(y=bounds_log[0, 1], linestyle='--')

#     # axes[5].axhline(y=bounds_hcw[0, 0], linestyle='--')
#     # axes[5].axhline(y=bounds_hcw[0, 1], linestyle='--')

#     axes[5].axhline(y=bounds_th[0, 0], linestyle='--')
#     axes[5].axhline(y=bounds_th[0, 1], linestyle='--')

#     # Py
#     plotting.plot_burn_y_error(axes[6], data_burn)
#     axes[6].axhline(y=bounds_log[1, 0], linestyle='--')
#     axes[6].axhline(y=bounds_log[1, 1], linestyle='--')

#     # axes[6].axhline(y=bounds_hcw[1, 0], linestyle='--')
#     # axes[6].axhline(y=bounds_hcw[1, 1], linestyle='--')

#     axes[6].axhline(y=bounds_th[1, 0], linestyle='--')
#     axes[6].axhline(y=bounds_th[1, 1], linestyle='--')

#     # Pz
#     plotting.plot_burn_z_error(axes[7], data_burn)
#     axes[7].axhline(y=bounds_log[2, 0], linestyle='--')
#     axes[7].axhline(y=bounds_log[2, 1], linestyle='--')

#     # axes[7].axhline(y=bounds_hcw[2, 0], linestyle='--')
#     # axes[7].axhline(y=bounds_hcw[2, 1], linestyle='--')

#     axes[7].axhline(y=bounds_th[2, 0], linestyle='--')
#     axes[7].axhline(y=bounds_th[2, 1], linestyle='--')
    
# burn_plotting(data_burn, bounds)

In [26]:
def burn_plotting(data_burn, bounds):
    fig, axes = plt.subplots(6, 1, figsize=(10, 10))
    axes[0].set_title(f'Burn Phase')
    colors = ['r','g','b']


    # Vx
    plotting.plot_burn_delta_vx_error(axes[0], data_burn)
    axes[0].axhline(y=bounds_log[3, 0], linestyle='--')
    axes[0].axhline(y=bounds_log[3, 1], linestyle='--')



    #V y
    plotting.plot_burn_delta_vy_error(axes[1], data_burn)
    axes[1].axhline(y=bounds_log[4, 0], linestyle='--')
    axes[1].axhline(y=bounds_log[4, 1], linestyle='--')


    # Vz
    plotting.plot_burn_delta_vz_error(axes[2], data_burn)
    axes[2].axhline(y=bounds_log[5, 0], linestyle='--')
    axes[2].axhline(y=bounds_log[5, 1], linestyle='--')

   
    # Px
    plotting.plot_burn_x_error(axes[3], data_burn)
    axes[3].axhline(y=bounds_log[0, 0], linestyle='--')
    axes[3].axhline(y=bounds_log[0, 1], linestyle='--')

   
    # Py
    plotting.plot_burn_y_error(axes[4], data_burn)
    axes[4].axhline(y=bounds_log[1, 0], linestyle='--')
    axes[4].axhline(y=bounds_log[1, 1], linestyle='--')

   

    # Pz
    plotting.plot_burn_z_error(axes[5], data_burn)
    axes[5].axhline(y=bounds_log[2, 0], linestyle='--')
    axes[5].axhline(y=bounds_log[2, 1], linestyle='--')


    
burn_plotting(data_burn, bounds)
plt.savefig("3h-05thrust-10dist.pdf", format="pdf")
plt.show()


NameError: name 'data_burn' is not defined